# Project Dallmayr

## Participants
- Lilian Do Khac
- Daniel Thuerck
- Boro Sofranac
- Marc Mezger

## Explanation
This is the Juypter notebook to present our results. There is always a Markdown Cell explaning that is going to happen and then the code.



# Downloading of the Project Files
In this step the Pickle Files are downloaded




In [ ]:
# First Download the files
import os
import urllib.parse
import urllib.request

# Create 'downloaded' directory if it does not exist
if not os.path.exists("downloaded"):
    os.makedirs("downloaded")

with open("data/hessenbox_link_list.txt") as file:
    for line in file:
        # Skip the lines that start with '#'
        if line[0] == "#":
            continue

        # Remove newline character from the end of line
        line = line.strip()

        # Download the file
        filename = os.path.basename(line)

        urllib.request.urlretrieve(urllib.parse.quote(line, safe=":/"), "downloaded/" + filename)


## Convert the Pickle to JSON

In [ ]:
import json
from pathlib import Path
from typing import List

import pandas as pd


def load_pickle_file(file_path: Path) -> pd.DataFrame:
    return pd.read_pickle(file_path)


def process_directory(directory: Path, output_directory: Path) -> int:
    files_list: List[Path] = [f for f in directory.iterdir() if f.is_file()]

    for file_path in files_list:
        df = load_pickle_file(file_path)
        df = df[["paper_title","publication_date", "plainpagefulltext"]]
        # Convert publication_date to string in ISO format
        df["publication_date"] = df["publication_date"].dt.strftime("%Y-%m-%d")
        df = df[df['paper_title'].str.contains('Kölnische Zeitung', case=False, na=False)]

        # Convert DataFrame to JSON
        json_data = df.to_dict(orient="records")

        # Create output file path
        output_file = output_directory / f"{file_path.stem}.json"

        # Write JSON to file
        with output_file.open("w", encoding="utf-8") as f:
            json.dump(json_data, f, ensure_ascii=False, indent=4)



if __name__ == "__main__":
    directory = Path("downloaded")
    output_directory = Path("jsons")

    # Create output directory if it doesn't exist
    output_directory.mkdir(exist_ok=True)

    process_directory(directory, output_directory)

## Cleaning the files

In this step the files are cleaned. For that purpose we used a selfhosted llm. The following variables were removed:     
"page_id",
    "provider_ddb_id",
    "provider",
    "zdb_id",
    "place_of_distribution",
    "language",
    "pagename",
    "thumbnail",
    "pagefulltext",
    "preview_reference"

Prompt for cleaning:
You are presented with some text in German from in between 1916 and 1945.
It is written in the old German of that time. Please go through the text
and adapt it to the modern german Language, removing spelling mistakes and
grammar mistakes where appropriate. Try to recover the text as faithfully as possible,
while minimizing the number of changes.
Replace any character from old German language with their equivalents of today, e.g.
"ſ" -> "s".

Only return the corrected text, do not add any prose or explanations on what you did.

Here is the text:
{data}


In [ ]:
import os.path
import pandas as pd
import simplejson as json
import pickle
from concurrent.futures import ThreadPoolExecutor

from typing import List

import ftfy
import openai
from dallmayr.prompts.prompt_loader import PromptLoader

class DatetimeEncoder(json.JSONEncoder):

  def default(self, obj):
    try:
      return super().default(obj)
    except TypeError:
      return str(obj)

class Cleaner:

  to_remove: List[str] = [
    "page_id",
    "provider_ddb_id",
    "provider",
    "zdb_id",
    "place_of_distribution",
    "language",
    "pagename",
    "thumbnail",
    "pagefulltext",
    "preview_reference"
  ]

  @staticmethod
  def pd2json(data_path, files, year, paper_map):

    # load all datafiles and concat all of them as
    full_data = {t[1]: [] for t in paper_map.items()}
    for file in files:
      file_path = os.path.join(data_path, file)
      df = pd.read_pickle(file_path)
      json_data = df.to_dict(orient='records')

      for record in json_data:
        for key in Cleaner.to_remove:
          del record[key]

        for p_title, t_key in paper_map.items():
          if p_title in record['paper_title']:
            full_data[t_key].append(record)

    # Save the JSON data to a file
    for _, t_key in paper_map.items():
      with open(f'{data_path}/json/{t_key}_{year}.json', 'w') as f:
        json.dump(full_data[t_key], f, cls=DatetimeEncoder)

  @staticmethod
  def json2cleanjson(file_path, llm_model, llm_host):
    # Initialize the OpenAI client
    client_llm = openai.OpenAI(
      api_key = "None",
      base_url = f"http://{llm_host}:5000/v1"
    )
    client_embedding = openai.OpenAI(
      api_key = "None",
      base_url = f"http://{llm_host}:5002/"
    )

    # import data
    with open(file_path, 'r', encoding="utf-8") as f:
      full_data = json.load(f)

    def clean_record(record):
      # fix encoding mistakes etc
      full_text = ftfy.fix_text(record['plainpagefulltext'])

      # cap so we don't exceed LLM size
      last_ix = min(16384, len(full_text))
      full_text = full_text[0:last_ix]

      # use a small LLM to clean the text
      prompt = PromptLoader.load("clean_ocr_data").format(
        data = full_text)

      # Make a completion call
      response = client_llm.chat.completions.create(
        model = llm_model,
        messages = [
          {"role": "system", "content": "You are a helpful assistant."},
          {"role": "user", "content": prompt},
        ],
        max_tokens=len(full_text),
        temperature=0.9
      )

      # Extract the cleaned text from the response
      cleaned_text = response.choices[0].message.content
      record['cleantext'] = cleaned_text

      # Compute the embedding
      response = client_embedding.embeddings.create(
        model = "nomic-ai/nomic-embed-text-v1",
        input = full_text
      )

      # Extract the embedding vector from the response
      embedding = response.data[0].embedding
      record['embedding'] = embedding

      return record

    with ThreadPoolExecutor(max_workers=10) as executor:
      # Submit the transform_item function for each item in the list
      results = list(executor.map(clean_record, full_data))

    # output cleaned data
    clean_file_path = file_path.replace('.json', '_clean.json')
    with open(clean_file_path, 'w', encoding="utf-8") as f:
      json.dump(results, f, cls=DatetimeEncoder)
    
    

## Starting of the Cleaning

In [ ]:
import os.path
import time

from dallmayr.cleaning.cleaner import Cleaner

SCRIPT_PATH = os.path.dirname(os.path.realpath(__file__))
DATA_PATH = os.path.join(SCRIPT_PATH, "..", "..", "data", "downloaded")
JSON_PATH = os.path.join(SCRIPT_PATH, "..", "..", "data", "downloaded", "json")

def main():

  
  files = os.listdir(JSON_PATH)

  for file in files:
    print(f"Cleaning file: {file}")

    start = time.time()
    Cleaner.json2cleanjson(os.path.join(JSON_PATH, file), "llama3-small", 
      "localhost")
    end = time.time()

    print(f"> Completed in {end - start} seconds.")

    exit(1)

if __name__ == '__main__':
  main()

## Summarize the newspaper articles
In the next steps the articles are summorized using Googles Gemini Flash Model. This allows to reduce the amount of text while still keeping the important information. This was done for all of the selected newspaper articles. Which were always the first page of the newspapers. We selected only three newspapers to allow for a faster development with lower costs for large language models.

In [ ]:
import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory
from loguru import logger
from tqdm import tqdm

load_dotenv()
safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}


llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", temperature=0, max_tokens=200, timeout=None, max_retries=50, safety_settings=safety_settings
)  # .with_structured_output(result_schema)

# load the data
sueddeutsche = pd.read_pickle("exported_data/suddeutsche.pkl")



messages = [
    (
        "human",
        """Input: {{text}}

You will generate topics and dense summaries of the topics of the above collection of articles.

Execute the following 2 steps.

Step 1. Identify informative topics from the collection of Articles.
Step 2. Write a summaries of identical length which covers every entity.

An Entity is:

Specific: descriptive yet concise (5 words or fewer).
Faithful: present in the Articles.
Anywhere: located anywhere in the Articles.

Guidelines:

The summary should be long (4-5 sentences, ~80 words)
The summaries should become highly dense and concise yet self-contained, e.g., easily understood without the Articles.
Replace old German characters (e.g. ſ) with modern German characters.

Remember, use the exact same number of words for each summary.

Answer in German with a JSON. The JSON should be a list of dictionaries whose keys are "Topic", "Keywords", "Persons", "Organizations", "Sentiment", "Objects", "Technology" and "Summary".""",
    ),
]


template = ChatPromptTemplate.from_messages(messages, template_format="jinja2")

sueddeutsche = sueddeutsche.sort_values(by="publication_date")

sueddeutsche = sueddeutsche.iloc[::50, :]

chain = template | llm
result_list = []


def chunker(seq, size):
    for pos in range(0, len(seq), size):
        yield seq.iloc[pos : pos + size]


chunk_size = 30
for i in tqdm(chunker(sueddeutsche, chunk_size)):
    texts = [{"text": t.plainpagefulltext} for t in i.itertuples()]
    
    # save the original text as txt file
    for t in i.itertuples():
        with open(
            f"original_sued/{t.publication_date.year}_sued_{t.publication_date.day}-{t.publication_date.month}-{t.publication_date.year}_{t.pagenumber!s}.txt", "w"
        ) as f:
            f.write(t.plainpagefulltext)



# Embedd and classify the sentiment

Now where are Embedding the Texts with Google Text Embedding 004 and the clustering setting. 
Clustering embedded texts involves grouping similar texts together based on their meaning, as represented by their embeddings. Embeddings are numerical vector representations of text where semantically similar texts have vectors that are close together in the vector space. This allows for more nuanced clustering than simply looking at keywords or word counts. In addition we classify the sentiment.

In [ ]:
import json
from pathlib import Path

import numpy as np
from langchain_google_vertexai import VertexAIEmbeddings

# Initialize the a specific Embeddings Model version
embeddings = VertexAIEmbeddings(model_name="text-embedding-004")


output_directory = Path("embeddings")
output_directory.mkdir(exist_ok=True)  # Ensure the output directory exists
input_directory = Path("jsons")  # Define your input directory path

def main():

    # Loop over the JSON files in the input folder
    for f in input_directory.glob("*.json"):
        # Load JSON file
        with open(f, "r", encoding="utf-8") as ff:
            data = json.load(ff)

        # Loop over the objects in the JSON file
        for idx, json_obj in enumerate(data):
            # Generate embeddings for each object
            embeddings = embeddings.embed_text(prompt=json_obj["plainpagefulltext"], task="clustering")
            # Convert embeddings to NumPy array
            embeddings_array = np.array(embeddings)
            # Save embeddings to a compressed NumPy file with a unique name
            output_file = output_directory / f"{f.stem}_{idx}.npz"
            np.savez_compressed(output_file, embeddings=embeddings_array)

if __name__ == "__main__":
    main()

## Clustering
Afterwards we are clustering with KMeans

In [ ]:
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans
from pathlib import Path
import datetime
import json
import pandas as pd
import cohere
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import seaborn as sns
from kneed import KneeLocator
from sklearn.decomposition import PCA

k = 10

def cluster_embeddings(embeddings, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(embeddings)
    cluster_labels = kmeans.labels_
    return cluster_labels

def main():
    # Embedding of texts
    
    files = list(Path("embeddings_sentiment").rglob("*.json"))

    file_contents = []
    embeddings = []
    years = []
    newspaper = []
    days = []
    months = []
    texts=[]
    dates = []
    sentiments = []
    for f in files:
        with f.open() as file:
            content = json.load(file)
            file_contents.append(content)
            for c in content:
                years.append(int(f.name.split("_")[0]))
                newspaper.append(f.name.split("_")[1])
                days.append(int(f.name.split("_")[2].split("-")[0]))
                months.append(int(f.name.split("_")[2].split("-")[1]))
                texts.append(c["content"])
                embeddings.append(c["embedding"])
                sentiments.append(c["sentiment"])
                dates.append(datetime.datetime(int(f.name.split("_")[0]),int(f.name.split("_")[2].split("-")[1]),int(f.name.split("_")[2].split("-")[0] )))

    data = {"day": days, "month":months, "year": years, "newspaper": newspaper, "embeddings": embeddings, "text": texts, "date": dates, "sentiment": sentiments}

    df = pd.DataFrame(data)
    df = df[(df['year'] >= 1914) & (df['year'] <= 1919)]
    df = df.drop(df[~(df['sentiment'] == 'neutral')].index)
    #df = df.drop(df[~(df['year'] == 1939)].index)
    #df.drop(df[~(df['year'] == 1939)].index)

    # Clustering of embeddings
    cluster_labels = cluster_embeddings(list(df["embeddings"]), k)

    print("asdf")
    #df1.to_excel("results.xlsx")
    df["cluster labels"] = cluster_labels
    df.to_csv("results.csv", sep=";")

    # ------------------------------------------------------------- Visualization 1
    list_of_arrays = []

    for idx, row in df.iterrows():
        list_of_arrays.append(np.array(row["embeddings"]))

    my_array = np.array(list_of_arrays)

    pca = PCA(n_components=3)
    pca.fit(my_array) 
    X_pca = pca.transform(my_array) 

    total_var = pca.explained_variance_ratio_.sum() * 100

    fig = px.scatter_3d(
        X_pca,
        #x=df["newspaper"], y=df["sentiment"], z=df["year"],  
        x=0, y=1, z=2, 
        color=cluster_labels,
        opacity=0.5,
        title=f'Neutral Sentiment 1914-1919',
        labels=cluster_labels,
        #df["sentiment"]
    )
    fig.update_traces(marker_size = 4)
    fig.show()

if __name__ == "__main__":
    main()

## Summarization of the Reflection with Google Gemini


In [ ]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
from sklearn.cluster import KMeans
from pathlib import Path
import datetime
import json
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from loguru import logger

k = 10


def cluster_embeddings(embeddings, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(embeddings)
    cluster_labels = kmeans.labels_
    return cluster_labels


def pre():
    # Embedding of texts

    files = list(Path("json/embeddings_sentiment").rglob("*.json"))

    file_contents = []
    embeddings = []
    years = []
    newspaper = []
    days = []
    months = []
    texts = []
    dates = []
    titles = []
    sentiments = []
    for f in files:
        with f.open() as file:
            content = json.load(file)
            file_contents.append(content)
            for c in content:
                years.append(int(f.name.split("_")[0]))
                newspaper.append(f.name.split("_")[1])
                days.append(int(f.name.split("_")[2].split("-")[0]))
                months.append(int(f.name.split("_")[2].split("-")[1]))
                texts.append(c["content"])
                embeddings.append(c["embedding"])
                sentiments.append(c["sentiment"])
                titles.append(c["title"])
                dates.append(datetime.datetime(int(f.name.split("_")[0]), int(f.name.split("_")[2].split("-")[1]), int(f.name.split("_")[2].split("-")[0])))

    data = {"day": days, "month": months, "year": years, "newspaper": newspaper, "embeddings": embeddings, "text": texts, "date": dates, "sentiment": sentiments, "title": titles}

    df = pd.DataFrame(data)
    df = df[(df["year"] >= 1914) & (df["year"] <= 1919)]
    df = df.drop(df[~(df["sentiment"] == "positive")].index)
    cluster_labels = cluster_embeddings(list(df["embeddings"]), k)

    # df1.to_excel("results.xlsx")
    df["cluster labels"] = cluster_labels
    return df



genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-pro-002")
prompt = """
How did German Newspapers reflect postive sentiments about technological advancements and what factors influenced these perceptions? Newspaper articles:
"""


def gen(text: str):
    response = model.generate_content(prompt+text)
    return response.text

def main():
    df = pre()
    results = []

    for i in range(10):
        # group by cluster
        cluster = df[df["cluster labels"] == i]

        contents = []
        for idx, row in cluster.iterrows():
            contents.append("Titel:" + row["title"] + "\n" + "Text" + row["text"] + "\n")

        text = " ".join(contents)

        result = gen(text)

        logger.info(f"Result for cluster {i}: {result}")
        results.append(result)


    # save results as text with cluster number
    with open("results.txt", "w") as f:
        for i, r in enumerate(results):
            f.write(f"Cluster {i}:\n")
            f.write(r)
            f.write("\n\n")

if __name__ == '__main__':
    main()

# Now letting the llm reflect how it changed from 1914 to 1944

In [ ]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()
import pandas as pd
from sklearn.cluster import KMeans
from pathlib import Path
import datetime
import json
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from loguru import logger


genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-1.5-pro-002")
prompt = """
Du bekommst zwei text inputs der erste ist ovn 1914-1919 und der zweite von 1938-1944. 
How did German Newspapers reflect postive sentiments about technological advancements and what factors influenced these perceptions? Newspaper articles:
"""


def gen(text: str):
    response = model.generate_content(prompt + text)
    return response.text


def main():
    # open the txt files 1914 and 1944 and sent it to the model
    file1914 = Path("results19141918-positive.txt")
    file1944 = Path("results1944-positive.txt")

    with file1914.open() as f:
        text1914 = f.read()

    with file1944.open() as f:
        text1944 = f.read()


    # combine the two texts
    combined_text = "die artikel von 1914-1919"+text1914 + "\n\n\n" + "die artikel von 1938-1945"+text1944

    results = gen(combined_text)

if __name__ == "__main__":
    main()


The results Postive Reflection

Zwischen 1914-1919 und 1938-1945 verändert sich die Darstellung und Wahrnehmung technologischer Fortschritte in deutschen Zeitungen deutlich.  Hier eine Zusammenfassung der wichtigsten Unterschiede:

**1914-1919 (Erster Weltkrieg und unmittelbare Nachkriegszeit):**

* **Indirekte positive Darstellung:** Technologie wird nicht direkt gepriesen, sondern im Kontext von positiven Folgen wie internationale Kooperation, wirtschaftliche Erholung und nationale Einheit.  Der Fokus liegt auf den *Ergebnissen* des technologischen Fortschritts, nicht auf der Technologie selbst.
* **Diplomatie und Wirtschaft im Vordergrund:**  Die Artikel betonen die Rolle von Technologien wie Telegraf, Zug und Dampfschiff bei der Ermöglichung von Diplomatie, Handel und internationalen Verträgen.
* **Propaganda und Moralerhaltung:** Die positive Darstellung dient der Moralerhaltung und der Rechtfertigung der Kriegsanstrengungen.  Negative Aspekte der Technologie, insbesondere ihre zerstörerische Kraft im Krieg, werden heruntergespielt.
* **Implizite Anerkennung wissenschaftlichen Fortschritts:**  In einigen Artikeln wird die Bedeutung wissenschaftlicher Forschung und Entwicklung angedeutet, jedoch ohne detaillierte technische Beschreibungen.

**1938-1945 (Zeit des Nationalsozialismus und Zweiter Weltkrieg):**

* **Direkte und überschwängliche positive Darstellung:** Technologischer Fortschritt, insbesondere in Bereichen wie Luftfahrt, Automobilbau und Kommunikation, wird explizit gepriesen und als Beweis deutscher Überlegenheit dargestellt.
* **Nationalismus und Propaganda:** Die Artikel sind stark nationalistisch geprägt und dienen der Propaganda des NS-Regimes.  Technologische Errungenschaften werden als Zeichen nationaler Stärke und Größe inszeniert.
* **Fokus auf Rekorde und Siege:**  Die Berichterstattung konzentriert sich auf Rekorde, Siege in Rennen und sportliche Triumphe, die die deutsche technische Leistungsfähigkeit demonstrieren sollen.
* **Verherrlichung militärischer Technologie:** Im Kontext des Zweiten Weltkriegs wird die militärische Technologie, insbesondere die Luftwaffe und U-Boote, glorifiziert.
* **Ausblendung negativer Aspekte und der Konkurrenz:**  Die Leistungen anderer Nationen werden heruntergespielt, und negative Aspekte oder Herausforderungen der deutschen Technologie werden ausgeblendet.
* **Infrastruktur und Autarkie:** Technologischer Fortschritt wird mit der Stärkung der Infrastruktur und dem Streben nach Autarkie verknüpft.

**Zusammenfassend lässt sich sagen:**

Die Wahrnehmung technologischen Fortschritts wandelt sich von einer indirekten, kontextbezogenen und propagandistisch geprägten Darstellung im Ersten Weltkrieg zu einer direkten, überschwänglichen und nationalistisch aufgeladenen Verherrlichung im Nationalsozialismus. Der Fokus verschiebt sich von den positiven Folgen der Technologie auf die Technologie selbst, insbesondere im militärischen Bereich.  Die Berichterstattung wird zunehmend propagandistisch und dient der Stärkung des NS-Regimes.



Negative Reflection
Die größte Veränderung zwischen 1914-1919 und 1938-1945 ist der **Wechsel vom Ersten zum Zweiten Weltkrieg und der damit einhergehende Aufstieg des Nationalsozialismus in Deutschland**.  Diese Veränderung prägt die Sichtweise auf technologischen Fortschritt in den Zeitungsartikeln fundamental.

**1914-1919:**

* **Desillusionierung durch den Krieg:**  Die Erfahrung des Ersten Weltkriegs, insbesondere die verheerenden Folgen neuer Waffentechnologien wie Giftgas und Maschinengewehre, führten zu einer Ernüchterung über den technologischen Fortschritt.  Technologie wurde mit Leid, Zerstörung und dem Scheitern der Kriegsziele assoziiert.
* **Soziale und wirtschaftliche Folgen:** Die Kriegswirtschaft, Ressourcenknappheit und soziale Umwälzungen wurden ebenfalls mit Technologie in Verbindung gebracht, allerdings weniger direkt.  Es gab Ängste vor den Folgen der Industrialisierung und der Veränderung der Arbeitswelt.
* **Propaganda und Medien:** Die zunehmende Bedeutung von Massenmedien und Propaganda wurde als zweischneidiges Schwert wahrgenommen. Technologie ermöglichte die Verbreitung von Informationen, aber auch deren Manipulation.

**1938-1945:**

* **Instrumentalisierung der Technologie:** Unter dem Nationalsozialismus wurde Technologie gezielt für die Kriegsführung und die Durchsetzung der Ideologie instrumentalisiert.  Technologischer Fortschritt wurde vor allem im militärischen Bereich vorangetrieben, während zivile Innovationen in den Hintergrund traten.
* **Kontrolle und Zensur:**  Die Nazis kontrollierten die Medien und unterdrückten kritische Stimmen.  Technologien der Kommunikation wurden für Propaganda und die Verbreitung der NS-Ideologie genutzt.
* **Wirtschaftliche Regulierung:** Die Wirtschaft wurde streng reguliert und auf den Krieg ausgerichtet.  Technologischer Fortschritt wurde durch Ressourcenknappheit und die Fokussierung auf Rüstungsproduktion gehemmt.
* **Feindbild und Propaganda:** Technologische Überlegenheit der Alliierten wurde heruntergespielt oder als barbarisch dargestellt.  Deutsche "Wunderwaffen" wurden propagandistisch hervorgehoben, um die Moral zu stärken.

**Zusammenfassend lässt sich sagen:**

Während im Ersten Weltkrieg eine generelle Desillusionierung über den technologischen Fortschritt herrschte, der mit Zerstörung und Leid verbunden wurde, wurde Technologie im Zweiten Weltkrieg unter den Nazis gezielt instrumentalisiert und kontrolliert.  Die Propaganda nutzte die Technologie zur Verbreitung der Ideologie und zur Manipulation der öffentlichen Meinung.  Die negative Wahrnehmung von Technologie konzentrierte sich dabei vor allem auf die technologische Überlegenheit der Feinde und die damit verbundene Bedrohung.  Die Kontrolle und Lenkung der Technologie durch das NS-Regime hemmte zudem den zivilen technologischen Fortschritt.
